# Spark GraphFrames

In [ ]:
import numpy as np

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [ ]:
from pyspark.sql.types import *
from graphframes import *

In [ ]:
import graphframes

**Notes on installing GraphFrames**

I needed to put the GraphFrames jar file into the Spark jars director for it to work.

```bash
cd /usr/local/spark/jars/
wget http://dl.bintray.com/spark-packages/maven/graphframes/graphframes/0.7.0-spark2.4-s_2.11/graphframes-0.7.0-spark2.4-s_2.11.jar
```

In [ ]:
import networkx as nx
from networkx.drawing.nx_pydot import graphviz_layout

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
G = nx.les_miserables_graph()

In [ ]:
plt.figure(figsize=(14, 6))
nx.draw_kamada_kawai(
    G, 
    with_labels=True, 
    node_size=500, 
    node_color='lightblue' 
)

In [ ]:
edges = [(a, b, np.random.randint(1, 4)) for (a, b) in nx.edges(G)]

In [ ]:
nodes = [(name, name, np.random.randint(15, 60)) for name in nx.nodes(G)]

In [ ]:
edges[:3]

In [ ]:
nodes[:3]

## Creation of a GraphFrame

- Need to provide a DataFrame of vertices, and a DataFrame of edges

### Vertices must contain an id column

In [ ]:
v = spark.createDataFrame(nodes, ['id', 'name', 'age'])

### Edges must have src and dst columns that contain vertex ids

In [ ]:
e = spark.createDataFrame(edges, ['src', 'dst', 'wt'])

In [ ]:
g = GraphFrame(v, e)

## Basic information

In [ ]:
g

In [ ]:
g.vertices.show(5)

In [ ]:
g.edges.show(5)

## Search

In [ ]:
from pyspark.sql.functions import col

In [ ]:
g.vertices.show(5)

### Can use regular DataFrame functions

In [ ]:
g.vertices.filter(col('name').startswith('C')).show(5)

In [ ]:
g.edges.groupBy('wt').count().show()

## Motifs

In [ ]:
g.find('(a)').show(5)

In [ ]:
g.find('(a)-[e]->(b)').show(5, truncate=False)

In [ ]:
g.find('(a)-[e1]->(b); (b)-[e2]->(c)').select('e1', 'e2').show(5, truncate=False)

In [ ]:
g.find('(a)-[]->(b); !(b)-[]->(a)').show(5, truncate=False)

In [ ]:
g.find('(a)-[]->(b); (b)-[]->(a)').show(5, truncate=False)

## Extracting subgraphs

In [ ]:
g1 = g.filterVertices("age > 30").filterEdges('wt < 3').dropIsolatedVertices()

In [ ]:
g.vertices.count()

In [ ]:
g1.vertices.count()

## Pathfinding

In [ ]:
path = g.bfs("name = 'Cosette'", "age > 30")

In [ ]:
path.show(truncate=False)

In [ ]:
path = g.bfs("name = 'Cosette'", "age = 50")

In [ ]:
path.show(truncate=False)

In [ ]:
paths = g.shortestPaths(landmarks = ['Cosette'])

In [ ]:
path.show(truncate=False)

## Centrality

In [ ]:
g.degrees.show(5)

In [ ]:
g.indegrees.show(5)

In [ ]:
g.outdegrees.show(5)

In [ ]:
res = g.pageRank(resetProbability=0.15, maxIter=5)

In [ ]:
res.show(truncate=False)

In [ ]:
res = g.pageRank(resetProbability=0.15, maxIter=5, sourceIds=['Cosette', 'Napolean'])

In [ ]:
res.show(truncate=False)

## Community detection

In [ ]:
sc = spark.sparkContext
sc.setCheckpointDir("checkpoint")

In [ ]:
g1 = g.filterEdges('wt = 1')

In [ ]:
res = g1.connectedComponents()

In [ ]:
res.show(truncate=False)

In [ ]:
res = g1.stronglyConnectedComponents(maxIter=5)

In [ ]:
res.show(truncate=False)

In [ ]:
res = g.triangleCount()

In [ ]:
res.show(truncate=False)

In [ ]:
res = g.labelPropagation(maxIter=5)

In [ ]:
res.show(truncate=False)